# TP4 : Reconnaissance de mots isolés par programmation dynamique (DTW)

Gonzalez Jean-Loup :
Déposer le notebook sur moodle à la fin de la séance.

In [79]:
import numpy as np
from math import floor, inf, dist
import scipy.io.wavfile as wav
import os
import matplotlib.pyplot as plt

# CONSTANTES
q = 16
taille_fenetre = 1024
nbe_coef = 16
rep_ref = 'SIGNAL/REF'
rep_obs = 'SIGNAL/OBS'
rep_mfcc = 'MFCC'


# Réutilisation de fonctions pour la lecture et la paramétrisation

In [80]:
# Réutiliser la fonction lecture permettant de lire un fichier son et de connaître ses caractéristiques 
# (fréquence d’échantillonnage, durée).

# LECTURE d'un fichier Wave
def lecture(fichier, nb_bits):
    f_ech,signal=wav.read(fichier)
    signal=signal/((2**nb_bits)-1)
    #calcul de la duree de l'audio en s
    duree=len(signal)/f_ech
    return signal,f_ech,duree

#test
signal1,f_ech1,duree1=lecture('SIGNAL/OBS/alpha.wav',16)
print(f_ech1)

16000


In [81]:
# Réutiliser la fonction parametrisation qui calcule les paramètres acoustiques
# (coefficients cepstraux) d’un fichier sonore sur des fenêtres de taille  "taille_fenetre"   avec un
# recouvrement de moitié .
    
# Calcul de "nbe_coef" coefficients cepstraux

#nbe_coef : nbe de MFCC 8,12,16,30
#taille_fenetre : 1024, pouissances de 2
def parametrisation(signal, taille_fenetre, nbe_coef):
    # Initialisation de la matrice résultat
    recouvrement = floor(taille_fenetre / 2)
    nb_fen = floor((np.size(signal) - taille_fenetre) / recouvrement) + 1
    mfcc = np.zeros((nb_fen, nbe_coef))

    # Calcul des MFCC
    for fen in range(nb_fen):
        p = fen * recouvrement
        spectre = abs(np.fft.fft(np.multiply(signal[p:p + taille_fenetre], np.hamming(taille_fenetre))))
        cepstre = np.fft.fft(np.log(spectre))
        cc = cepstre[1:nbe_coef+1].real
        mfcc[fen, :] = cc

    return mfcc

In [97]:
# Modifier votre fonction   parametrisation_total  (utilisant la fonction   parametrisation  )
# afin qu’elle permette de calculer les paramètres pour l’ensemble des fichiers d’un répertoire
# rep_wav   et de stocker les résultats dans un répertoire   rep_mfcc. Cette fonction renvoie
# la liste des noms de fichiers traités.
    
# Fonction de paramétrisation totale --> tous les fichiers du répertoire 'WAV/APP'
def parametrisation_total(nb_bits, taille_fenetre, nbe_coef, rep_wav, rep_mfcc):
    liste_noms_fichiers=[]
    for fichier in  os.listdir(rep_wav):
        nom_fichier=fichier.removesuffix('.wav')
        liste_noms_fichiers.append(nom_fichier)
        nom_fichier_a_lire = rep_wav + '/'+ nom_fichier+'.wav'
        nom_fichier_a_ecrire = rep_mfcc +'/' + nom_fichier+'.mfcc'
        signal_a_parametrer,f_ech,duree=lecture(nom_fichier_a_lire,nb_bits)
        mfcc_signal=parametrisation(signal_a_parametrer,taille_fenetre,nbe_coef)
        np.savetxt(nom_fichier_a_ecrire, mfcc_signal, fmt='%d')
    print("etiquetage total fini")
    return liste_noms_fichiers

liste_noms_fichiers_traites_obs= parametrisation_total(q,taille_fenetre,nbe_coef,'SIGNAL/OBS','SIGNAL/MFCC_OBS')
liste_noms_fichiers_traites_ref= parametrisation_total(q,taille_fenetre,nbe_coef,'SIGNAL/REF','SIGNAL/MFCC_REF')

etiquetage total fini
etiquetage total fini


# Programmation dynamique

In [134]:
# Ecrire une fonction dtw qui prend deux arguments en entrée : la matrice de coefficients
# cepstraux du signal à reconnaître (observation) et la matrice de coefficients cepstraux d'un
# signal de référence. Cette fonction renvoie le coût normalisé.
    
# Calcul de la DTW entre deux vecteurs
def dtw(path_mfcc_ref, path_mfcc_obs):
    # Lecture des 2 fichiers
    mfcc_ref = np.loadtxt(path_mfcc_ref)
    mfcc_obs = np.loadtxt(path_mfcc_obs)
    
    #initialisation
    nb_vect_mfcc_ref=len(mfcc_ref)
    nb_vect_mfcc_obs=len(mfcc_obs)
    cout_case=np.zeros((nb_vect_mfcc_obs,nb_vect_mfcc_ref))
    cout_case[0][0]=0
    
    #premiere ligne a infini
    for i in range(nb_vect_mfcc_obs):
        cout_case[i][0]=inf  
        
    #premiere colone a infini
    for j in range(nb_vect_mfcc_ref):
        cout_case[0][j]=inf
        
    #calcul du cout pour chaque case
    for i in range (nb_vect_mfcc_obs):
        for j in range(nb_vect_mfcc_ref):
            cout_case[i][j]=min(cout_case[i-1][j]+dist(mfcc_obs[i],mfcc_ref[j]),cout_case[i-1][j-1]+2*dist(mfcc_obs[i],mfcc_ref[j]),cout_case[i][j-1]+dist(mfcc_obs[i],mfcc_ref[j]))
            
    #calcul de la distance euclidienne finale      
    distance_euclidienne_finale= (cout_case[nb_vect_mfcc_obs-1][nb_vect_mfcc_ref-1])/(nb_vect_mfcc_obs+nb_vect_mfcc_ref)

    return distance_euclidienne_finale


In [129]:
#Ecrire une fonction   dtw_total   (utilisant la fonction   dtw  ) qui calcule la DTW sur toutes
# les observations (fichiers Wave) du répertoire   rep_obs   par rapport   toutes les références
# (fichiers MFCC) du répertoire   rep_ref. Cette fonction renvoie une matrice de coûts de taille
# nb_fichier_obs x nb_fichier_ref.

# DTW sur toutes les observations par rapport à toutes les références
def DTW_total(nb_bits, taille_fenetre, nbe_coef, rep_mfcc_ref, rep_wave_obs):
    list_score_par_obs=[]
    list_score_totale=[]
    #itere sur les obs en .wav
    for fichier_obs in os.listdir(rep_wave_obs):
        #itere sur les reference en .mfcc
        for fichier_ref in  os.listdir(rep_mfcc_ref):
            #recupere le chemin d'acces vers le dossier contenant l'observation .wav courante
            path_fichier_obs=rep_wave_obs+'/'+fichier_obs
            #on calcule les MFCC de l'obs .wav courante
            obs,_,_=lecture(path_fichier_obs,nb_bits)
            mfcc_obs=parametrisation(obs, taille_fenetre, nbe_coef)
            #on enregistre ces MFCC dans un répertoire
            nom_fichier_a_ecrire='SIGNAL/MFCC_OBS'+'/'+fichier_obs
            np.savetxt(nom_fichier_a_ecrire, mfcc_obs, fmt='%d')
            #on recupere le chemin d'acces vers ces dossiers
            path_mfcc_obs=nom_fichier_a_ecrire
            path_mfcc_ref=rep_mfcc_ref+'/'+fichier_ref
            #calcule score entre le fichier .wav et tous les .mfcc ref
            list_score_par_obs.append(dtw(path_mfcc_ref,path_mfcc_obs))
        list_score_totale.append(list_score_par_obs.copy())
    return list_score_totale
    
DTW_total(q,taille_fenetre,nbe_coef,'SIGNAL/MFCC_REF','SIGNAL/OBS')

[[248.92923353309646,
  256.1545610743815,
  334.7589686828021,
  216.55487191680683,
  262.77377148033673,
  251.20401437345288,
  223.40095953795677,
  262.41218202091915,
  224.26901517868947,
  253.8913188177755,
  281.20106711913274,
  273.89517194540076,
  262.39460830711977],
 [248.92923353309646,
  256.1545610743815,
  334.7589686828021,
  216.55487191680683,
  262.77377148033673,
  251.20401437345288,
  223.40095953795677,
  262.41218202091915,
  224.26901517868947,
  253.8913188177755,
  281.20106711913274,
  273.89517194540076,
  262.39460830711977,
  192.00775852142868,
  196.07240120138934,
  309.9339664777286,
  213.239056767178,
  222.19075626759485,
  211.5097267968473,
  219.57577428097844,
  235.16267882956058,
  200.51294665662266,
  216.95118593579278,
  198.73614644399257,
  189.02865171732236,
  232.3220563639106],
 [248.92923353309646,
  256.1545610743815,
  334.7589686828021,
  216.55487191680683,
  262.77377148033673,
  251.20401437345288,
  223.40095953795677,

# Affichages et tests

In [131]:
# Ecrire un programme principal qui lance les fonctions précédentes et affiche pour chaque
# observation (mot inconnu), le mot le plus probable.

# Initialisation
q = 16
taille_fenetre = 1024
nbe_coef = 16
rep_ref = 'SIGNAL/MFCC_REF'
rep_obs = 'SIGNAL/OBS'
rep_mfcc = 'SIGNAL/MFCC'


In [135]:
# Paramétrisation des fichiers références
liste_noms_fichiers_traites_obs= parametrisation_total(q,taille_fenetre,nbe_coef,'SIGNAL/OBS','SIGNAL/MFCC_OBS')
liste_noms_fichiers_traites_ref= parametrisation_total(q,taille_fenetre,nbe_coef,'SIGNAL/REF','SIGNAL/MFCC_REF')

# Test de la DTW

#test avec 2 fois le meme fichier : fait 0
print(dtw('SIGNAL/MFCC_OBS/alpha.mfcc','SIGNAL/MFCC_OBS/alpha.mfcc'))
#test avec 2 fichiers different, le OBS et le REF correspondant
print(dtw('SIGNAL/MFCC_REF/alpha.mfcc','SIGNAL/MFCC_OBS/alpha.mfcc'))



etiquetage total fini
etiquetage total fini
0.0
248.92923353309646


In [146]:
# DTW sur toutes les observations par rapport à chaque référence
matrice_DTW=DTW_total(q,taille_fenetre,nbe_coef,rep_ref,rep_obs)

# Affichage des couts
print(matrice_DTW)

#afficher le mot correspondant le plus probable
score_min=[]
for scores_mot in matrice_DTW:
    score_min.append(min(scores_mot))
    """TODO : pb matrice DTW, stack ! on dépasse les 13x13 (13 obs pour 13 ref)"""

[[248.92923353309646, 256.1545610743815, 334.7589686828021, 216.55487191680683, 262.77377148033673, 251.20401437345288, 223.40095953795677, 262.41218202091915, 224.26901517868947, 253.8913188177755, 281.20106711913274, 273.89517194540076, 262.39460830711977], [248.92923353309646, 256.1545610743815, 334.7589686828021, 216.55487191680683, 262.77377148033673, 251.20401437345288, 223.40095953795677, 262.41218202091915, 224.26901517868947, 253.8913188177755, 281.20106711913274, 273.89517194540076, 262.39460830711977, 192.00775852142868, 196.07240120138934, 309.9339664777286, 213.239056767178, 222.19075626759485, 211.5097267968473, 219.57577428097844, 235.16267882956058, 200.51294665662266, 216.95118593579278, 198.73614644399257, 189.02865171732236, 232.3220563639106], [248.92923353309646, 256.1545610743815, 334.7589686828021, 216.55487191680683, 262.77377148033673, 251.20401437345288, 223.40095953795677, 262.41218202091915, 224.26901517868947, 253.8913188177755, 281.20106711913274, 273.8951

In [2]:

# Affichage amélioré

# Ajouter à votre programme principal, un affichage des coûts entre une observation et chaque
# référence sous forme d'histogramme via la commande bar.
# Calculer le score de reconnaissance.

# Pour chaque observation, affichage des coûts (par rapport aux références) sous forme d'histogramme



In [3]:
# Affichage score final


In [ ]:
# Autres tests 
# Tests
# Bien  videmment, afin d’am liorer les r sultats, vous pourrez modifier :
# - le nombre de param tres (coefficients cepstraux) : nbe_coef,
# - la taille de la fen tre d’analyse : taille_fenetre,
# ET TESTER AVEC VOS PROPRES ENREGISTREMENTS !